In [1]:
#Importing the basic librarires
import numpy as np
import pandas as pd
import seaborn as sns
from classes.base import SMOTE, NaiveBayes

from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,6]

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

# Importing the dataset

raw_data = pd.read_csv(r'.\data\Spam.csv')

target = 'spam'
labels = ['Ham','Spam']
features = [i for i in raw_data.columns.values if i not in [target]]

original_df = raw_data.copy(deep=True)
display(raw_data.head())

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,...,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_hash,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.64,0.00,0.00,0.00,0.32,0.00,1.29,1.93,0.00,0.96,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,...,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.0,0.06,0.06,0.0,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [2]:
def calcular_probabilidades(dataset):
    # Crear un DataFrame para almacenar las probabilidades
    probabilidades = pd.DataFrame(columns=['Variable', 'Values', 'Probabilities'])

    # Iterar sobre las columnas del dataset (excepto la columna de la variable objetivo)
    for columna in dataset.columns[:-1]:
        # Obtener los valores únicos en la columna
        valores = dataset[columna].unique()

        # Calcular las probabilidades para cada valor
        for valor in valores:
            # Calcular la probabilidad para el valor actual
            probabilidad = dataset[dataset[columna] == valor]['spam'].mean()

            # Agregar las probabilidades al DataFrame
            probabilidades.loc[len(probabilidades)] = [columna, valor, probabilidad]

    return probabilidades

In [3]:
# Supongamos que tu dataset se llama "data"
probabilidades = calcular_probabilidades(raw_data)

# Imprimir las probabilidades
print(probabilidades)

                       Variable   Values  Probabilities
0                word_freq_make     0.00       0.330327
1                word_freq_make     0.21       0.500000
2                word_freq_make     0.06       0.350000
3                word_freq_make     0.15       0.692308
4                word_freq_make     0.05       0.407407
...                         ...      ...            ...
15089  capital_run_length_total   448.00       0.000000
15090  capital_run_length_total   390.00       0.000000
15091  capital_run_length_total   265.00       0.000000
15092  capital_run_length_total   422.00       0.000000
15093  capital_run_length_total  1789.00       0.000000

[15094 rows x 3 columns]


In [4]:
# Función para realizar inferencia basada en la estructura y las probabilidades
def inferencia_red(variable, evidencia, estructura, probabilidades):
    # Verificar si la variable tiene evidencia
    if variable in evidencia:
        return probabilidades.loc[probabilidades['Variable'] == variable]['Probabilities'].values[0][evidencia[variable]]
    
    # Obtener los padres de la variable
    parents = estructura.loc[estructura['Variable'] == variable]['Parents'].values[0]
    
    # Obtener las probabilidades condicionales de la variable
    probabilities = probabilidades.loc[probabilidades['Variable'] == variable]['Probabilities'].values[0]
    
    # Calcular la probabilidad de la variable utilizando la regla de Bayes
    probability = 1.0
    for parent in parents:
        parent_probability = inferencia_red(parent, evidencia, estructura, probabilidades)
        probability *= parent_probability
    
    # Obtener el índice correspondiente al valor de la variable
    value_index = estructura.loc[estructura['Variable'] == variable].index[0]
    
    return probability * probabilities[value_index]

# Ejemplo de uso de la función de inferencia
evidencia = {'Problema': 'No', 'Memoria': 'Buena', 'DiscoDuro': 'Bueno'}
resultado = inferencia_red('Solucion', evidencia, structure, probabilities)
print('La solución recomendada es:', probabilities.loc[probabilities['Variable'] == 'Solucion']['Values'].values[0][resultado])


NameError: name 'structure' is not defined

In [5]:
# Crear un DataFrame para almacenar la estructura de la red
structure = pd.DataFrame(columns=['Variable', 'Parents'])

# Agregar variables y sus padres a la estructura
structure.loc[0] = ['Problema', []]
structure.loc[1] = ['CPU', ['Problema']]
structure.loc[2] = ['Memoria', ['Problema']]
structure.loc[3] = ['DiscoDuro', ['Problema']]
structure.loc[4] = ['Graficos', ['Problema']]
structure.loc[5] = ['FuentePoder', ['Problema']]
structure.loc[6] = ['Solucion', ['Problema']]


In [6]:
# Crear un DataFrame para almacenar las probabilidades
probabilities = pd.DataFrame(columns=['Variable', 'Values', 'Probabilities'])

# Definir las probabilidades para la variable 'Problema'
probabilities.loc[0] = ['Problema', ['Si', 'No'], [0.3, 0.7]]

# Definir las probabilidades para la variable 'CPU'
probabilities.loc[1] = ['CPU', ['Buena', 'Mala'], [0.9, 0.1]]

# Definir las probabilidades para la variable 'Memoria'
probabilities.loc[2] = ['Memoria', ['Buena', 'Mala'], [0.8, 0.2]]

# Definir las probabilidades para la variable 'DiscoDuro'
probabilities.loc[3] = ['DiscoDuro', ['Bueno', 'Malo'], [0.95, 0.05]]

# Definir las probabilidades para la variable 'Graficos'
probabilities.loc[4] = ['Graficos', ['Buenos', 'Malos'], [0.85, 0.15]]

# Definir las probabilidades para la variable 'FuentePoder'
probabilities.loc[5] = ['FuentePoder', ['Buena', 'Mala'], [0.75, 0.25]]

# Definir las probabilidades para la variable 'Solucion'
probabilities.loc[6] = ['Solucion', ['Reparar', 'Reemplazar'], [0.8, 0.2]]


In [7]:
# Función para realizar inferencia basada en la estructura y las probabilidades
def inferencia_red(variable, evidencia, estructura, probabilidades):
    # Verificar si la variable tiene evidencia
    if variable in evidencia:
        return probabilidades.loc[probabilidades['Variable'] == variable]['Probabilities'].values[0][evidencia[variable]]
    
    # Obtener los padres de la variable
    parents = estructura.loc[estructura['Variable'] == variable]['Parents'].values[0]
    
    # Obtener las probabilidades condicionales de la variable
    probabilities = probabilidades.loc[probabilidades['Variable'] == variable]['Probabilities'].values[0]
    
    # Calcular la probabilidad de la variable utilizando la regla de Bayes
    probability = 1.0
    for parent in parents:
        parent_probability = inferencia_red(parent, evidencia, estructura, probabilidades)
        probability *= parent_probability
    
    # Obtener el índice correspondiente al valor de la variable
    value_index = estructura.loc[estructura['Variable'] == variable].index[0]
    
    return probability * probabilities[value_index]

# Ejemplo de uso de la función de inferencia
evidencia = {'Problema': 'No', 'Memoria': 'Buena', 'DiscoDuro': 'Bueno'}
resultado = inferencia_red('Solucion', evidencia, structure, probabilities)
print('La solución recomendada es:', probabilities.loc[probabilities['Variable'] == 'Solucion']['Values'].values[0][resultado])


TypeError: list indices must be integers or slices, not str